In [1]:
'''
 Prepared by Carolina B.

 This file consists of 3 parts:
 1) daily price data extraction of agricultural ETF's:
 Corn: CORN,
 Wheat: WEAT,
 Soybeans: SOYB 

 2) Cleaning of daily prices of futures market (extracted from Tradestation):
 corn_daily.csv, soybeans_daily.csv and wheat_daily.csv

 3) Concatenation and grouping of the daily data from each asset and compiled
 into one set of clean data 

 
'''
# Import libraries 
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime as dt


# PART 1 - Extraction and Cleaning of agricultural ETF's 

Source: ALPACA API

In [2]:
# Import json as per API documentation for Alpaca
import json


In [3]:
# Load .env enviroment variables
load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_api_key)

# Create the Alpaca API object

alpaca_trade = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

tickers = ["CORN", "WEAT", "SOYB"]
timeframe = "1D"

In [5]:
# Extract data for CORN, WEAT and SOYB with setstart and end date of 20 years from now
start_date = pd.Timestamp('2000-10-01', tz='America/Chicago').isoformat()
end_date = pd.Timestamp('2020-10-01', tz='America/Chicago').isoformat()

df_etf_data = alpaca_trade.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_etf_data.head()

CORN                                  SOYB       \
                            open   high      low    close  volume open high   
2010-06-09 00:00:00-04:00  25.12  25.25  25.1200  25.1500  1700.0  NaN  NaN   
2010-06-10 00:00:00-04:00  25.46  25.46  25.4599  25.4599   200.0  NaN  NaN   
2010-06-11 00:00:00-04:00  25.88  25.88  25.7913  25.7913   500.0  NaN  NaN   
2010-06-14 00:00:00-04:00  25.99  26.11  25.9900  26.1100  2247.0  NaN  NaN   
2010-06-15 00:00:00-04:00  26.24  26.24  25.9690  25.9690  6955.0  NaN  NaN   

                                           WEAT                        
                          low close volume open high low close volume  
2010-06-09 00:00:00-04:00 NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
2010-06-10 00:00:00-04:00 NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
2010-06-11 00:00:00-04:00 NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
2010-06-14 00:00:00-04:00 NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
2010-06-15 00:00:00-04:00 NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN

In [6]:
df_etf_data.dtypes


CORN  open      float64
      high      float64
      low       float64
      close     float64
      volume    float64
SOYB  open      float64
      high      float64
      low       float64
      close     float64
      volume    float64
WEAT  open      float64
      high      float64
      low       float64
      close     float64
      volume    float64
dtype: object

In [7]:
# drop 1 level on multiindex dataframe
df_etf_data.columns = df_etf_data.columns.droplevel()



In [8]:
df_etf_data.head(10)

,open,high,low,close,volume,open,high,low,close,volume,open,high,low,close,volume
2010-06-09 00:00:00-04:00,25.12,25.25,25.1200,25.1500,1700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-10 00:00:00-04:00,25.46,25.46,25.4599,25.4599,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-11 00:00:00-04:00,25.88,25.88,25.7913,25.7913,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-14 00:00:00-04:00,25.99,26.11,25.9900,26.1100,2247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-15 00:00:00-04:00,26.24,26.24,25.9690,25.9690,6955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-16 00:00:00-04:00,26.26,26.44,26.1999,26.3200,2200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-17 00:00:00-04:00,26.20,26.20,25.8200,26.0800,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-18 00:00:00-04:00,26.22,26.60,26.2199,26.3900,3125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-21 00:00:00-04:00,26.35,26.35,25.7000,25.9799,9500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-22 00:00:00-04:00,25.90,25.90,25.6600,25.6864,28700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_etf_data.tail(5)

,open,high,low,close,volume,open,high,low,close,volume,open,high,low,close,volume
2020-09-25 00:00:00-04:00,12.76,12.81,12.700,12.730,168089.0,15.4800,15.6300,15.46,15.5325,142710.0,5.45,5.4700,5.43,5.430,115228.0
2020-09-28 00:00:00-04:00,12.69,12.85,12.590,12.779,321282.0,15.5300,15.5300,15.40,15.4812,136101.0,5.42,5.5000,5.38,5.490,127091.0
2020-09-29 00:00:00-04:00,12.72,12.74,12.645,12.720,160606.0,15.4500,15.5500,15.35,15.4100,157959.0,5.49,5.5099,5.45,5.485,144401.0
2020-09-30 00:00:00-04:00,12.72,13.28,12.690,13.100,1179792.0,15.4400,15.9941,15.38,15.8409,429408.0,5.52,5.7900,5.52,5.710,327629.0
2020-10-01 00:00:00-04:00,13.25,13.30,13.200,13.230,140633.0,15.9101,15.9500,15.85,15.9000,125880.0,5.72,5.7200,5.64,5.660,191546.0


In [10]:
# This step is to remove columns: open, high, low and volume
df_etf_close_data = df_etf_data.drop(columns=["open","high","low","volume"])
#df_etf_close_data.head()
#this step is to change columns names to the ones defined on the list
df_etf_close_data.columns = tickers 
df_etf_close_data.head()


,CORN,WEAT,SOYB
2010-06-09 00:00:00-04:00,25.1500,NaN,NaN
2010-06-10 00:00:00-04:00,25.4599,NaN,NaN
2010-06-11 00:00:00-04:00,25.7913,NaN,NaN
2010-06-14 00:00:00-04:00,26.1100,NaN,NaN
2010-06-15 00:00:00-04:00,25.9690,NaN,NaN


In [11]:
# this step is to check and drop null's
df_etf_close_data.isnull().sum()


CORN      1
WEAT    341
SOYB    332
dtype: int64

In [12]:
df_etf_close_data.dropna(inplace =True)
df_etf_close_data.isnull().sum()

CORN    0
WEAT    0
SOYB    0
dtype: int64

In [13]:
# Reset index to remove time from date index
df_etf_close_data.reset_index(inplace=True)
#df_etf_close_data.head()
df_etf_close_data.rename(columns={"index":"date"},inplace=True)
#df_etf_close_data.head()
df_etf_close_data


,date,CORN,WEAT,SOYB
0,2011-09-19 00:00:00-04:00,45.650,24.5500,24.570
1,2011-09-20 00:00:00-04:00,45.540,25.0800,24.870
2,2011-09-21 00:00:00-04:00,44.800,24.3400,24.230
3,2011-09-22 00:00:00-04:00,42.990,24.0000,23.000
4,2011-09-23 00:00:00-04:00,42.450,23.0000,23.560
...,...,...,...,...
2245,2020-09-25 00:00:00-04:00,12.730,15.5325,5.430
2246,2020-09-28 00:00:00-04:00,12.779,15.4812,5.490
2247,2020-09-29 00:00:00-04:00,12.720,15.4100,5.485
2248,2020-09-30 00:00:00-04:00,13.100,15.8409,5.710


In [20]:
df_etf_close_data['date'] = pd.to_datetime(df_etf_close_data['date']).dt.date
df_etf_close_data


,date,CORN,WEAT,SOYB
0,2011-09-19,45.650,24.5500,24.570
1,2011-09-20,45.540,25.0800,24.870
2,2011-09-21,44.800,24.3400,24.230
3,2011-09-22,42.990,24.0000,23.000
4,2011-09-23,42.450,23.0000,23.560
...,...,...,...,...
2245,2020-09-25,12.730,15.5325,5.430
2246,2020-09-28,12.779,15.4812,5.490
2247,2020-09-29,12.720,15.4100,5.485
2248,2020-09-30,13.100,15.8409,5.710


In [24]:
# Reset the date as index

df_etf_close_data_final = df_etf_close_data.set_index('date')
df_etf_close_data_final.head()

,CORN,WEAT,SOYB
date,,,
2011-09-19,45.65,24.55,24.57
2011-09-20,45.54,25.08,24.87
2011-09-21,44.80,24.34,24.23
2011-09-22,42.99,24.00,23.00
2011-09-23,42.45,23.00,23.56


In [25]:
# final adjustment - change the columns to lowercase to meet convention
df_etf_close_data_final.columns = df_etf_close_data_final.columns.str.lower()

df_etf_close_data_final.head(2)

,corn,weat,soyb
date,,,
2011-09-19,45.65,24.55,24.57
2011-09-20,45.54,25.08,24.87


# Part 2 - Extract csv files for futures data

Source: csv file from Tradestation 

In [14]:
# Import relevant libraries
from pathlib import Path
import csv



In [15]:
# Read each csv file
corn_csv = Path("raw_data/corn_daily.csv")
df_corn = pd.read_csv(corn_csv, infer_datetime_format= True, parse_dates = True, index_col= "Date")
df_corn.drop(columns=["Time","Open","High","Low", "Vol","OI"], inplace=True)
df_corn.columns = ["corn"]
df_corn.head()

,corn
Date,
2000-10-26,213.75
2000-10-27,212.50
2000-10-30,212.00
2000-10-31,217.50
2000-11-01,217.50


In [29]:

def csvexatract_df (filepath,col_name):
    dest_name = pd.read_csv(filepath, infer_datetime_format= True, parse_dates = True, index_col= "Date")
    dest_name.drop(columns=["Time","Open","High","Low", "Vol","OI"], inplace=True)
    dest_name.columns = [col_name]
    return dest_name

cornf = csvexatract_df (corn_csv,"cornf")
cornf.head(2)



,cornf
Date,
2000-10-26,213.75
2000-10-27,212.50


In [30]:
soybean_csv = Path("raw_data/soybeans_daily.csv")
soybeanf_df = csvexatract_df(soybean_csv,"soybf")
soybeanf_df.head(2)

,soybf
Date,
2000-10-26,503.75
2000-10-27,500.50


In [31]:
wheat_csv = Path("raw_data/wheat_daily.csv")
wheatf_df = csvexatract_df(wheat_csv,"wheatf")
wheatf_df.head(2)

,wheatf
Date,
2000-10-26,313.0
2000-10-27,315.5


In [33]:
df_futures = pd.concat([cornf,soybeanf_df,wheatf_df], axis="columns", join="inner")
df_futures.head()

,cornf,soybf,wheatf
Date,,,
2000-10-26,213.75,503.75,313.00
2000-10-27,212.50,500.50,315.50
2000-10-30,212.00,502.75,312.25
2000-10-31,217.50,504.00,312.25
2000-11-01,217.50,505.00,311.75


# Part 3 - Prepare Combined dataframe and export to clean data folder



In [34]:
# Concatenate stock dataframe with futures dataframe

df_comb_stock_futures = pd.concat([df_etf_close_data_final,df_futures,],axis="columns", join="inner")
df_comb_stock_futures.head(2)

,corn,weat,soyb,cornf,soybf,wheatf
2011-09-19,45.65,24.55,24.57,692.25,1336.0,673.00
2011-09-20,45.54,25.08,24.87,690.25,1338.0,674.75


In [35]:
# Writeup final csv file and save it on clean data folder

outputpath = Path('clean_data/hist_daily_stock_fut_price.csv')

df_comb_stock_futures.to_csv(outputpath)